# GCAT - FB15k

### Content

* I. Initial

* II. Load Corpus

* III. Train GAT

* IV. Train Conv

* V. Evaluation

In [ ]:
from google.colab import drive
drive.flush_and_unmount()

Drive not mounted, so nothing to flush and unmount.


In [ ]:
rm -rf /content/*

# I. Initial

In [ ]:
%%bash
rm -rf GCAT
git clone https://github.com/hmthanh/GCAT.git

Cloning into 'GCAT'...
Checking out files: 100% (118/118), done.


In [ ]:
%%bash
rm -rf /content/data/
rm -rf /content/output/
rm -r /content/sample_data/

mv /content/GCAT/* /content/
rm -r /content/GCAT/

rm: cannot remove '/content/sample_data/': No such file or directory


Load Google Drive if you want to save model on Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/4AGQSSFc4RSI16slOnktLjaOX5Sc8lKdUi7k-ztKOslIwsCHFxeiDn0
Mounted at /content/drive


## 1. Config

In [ ]:
#@title Cấu hình cài đặt quá trình huấn luyện { run: "auto" }

#@markdown Siêu tham số

dataset = "FB15k" #@param ["WN18RR", "WN18", "FB15k", "FB15k-237"] {allow-input: false}
cuda = True #@param {type: "boolean"}
save_gdrive = True #@param {type: "boolean"}
data_folder = "./data" #@param {type: "string"}
output_folder="./output" #@param {type: "string"}
drive_folder="/content/drive/My Drive" #@param {type: "string"}

epochs_gat = 3000 #@param {type: "number"}
epochs_conv = 150 #@param {type: "number"}
weight_decay_gat = float(1e-5) #@param {type:"raw"}
weight_decay_conv = float(1e-5) #@param {type:"raw"}
pretrained_emb = False #@param {type: "boolean"}
embedding_size = 50 #@param {type: "slider", min: 20, max: 200}
lr = float(1e-3) #@param {type:"raw"}
get_2hop = True #@param {type: "boolean"}
use_2hop = True #@param {type: "boolean"}
partial_2hop = True #@param {type: "boolean"}

#@markdown --- Tham số cho mô hình GAT
batch_size_gat = 272115 #@param {type: "number"}
valid_invalid_ratio_gat = 2 #@param {type: "slider", min: 1, max: 10}
drop_GAT = 0.3  #@param {type:"raw"}
alpha = 0.2  #@param {type:"raw"}
entity_out_dim = [100, 200]  #@param {type:"raw"}
nheads_GAT = [2, 2]  #@param {type:"raw"}
margin = 1 #@param {type: "slider", min: 1, max: 10}

#@markdown --- Tham số cho mô hình ConvKB
batch_size_conv = 128  #@param {type: "slider", min: 16, max: 512, step:16}
alpha_conv = 0.2  #@param {type:"raw"}
valid_invalid_ratio_conv = 40
out_channels = 50 #@param {type: "slider", min: 50, max: 1000, step:50}
drop_conv = 0.3  #@param {type:"raw"}

from create_config import Config

config = Config(dataset=dataset, data_folder=data_folder,cuda=cuda,output_folder=output_folder,save_gdrive=save_gdrive,drive_folder=drive_folder,epochs_gat=epochs_gat,epochs_conv=epochs_conv,weight_decay_gat=weight_decay_gat,weight_decay_conv=weight_decay_conv,pretrained_emb=pretrained_emb,embedding_size=embedding_size,lr=lr,get_2hop=get_2hop,use_2hop=use_2hop,partial_2hop=partial_2hop,batch_size_gat=batch_size_gat,valid_invalid_ratio_gat=valid_invalid_ratio_gat,drop_GAT=drop_GAT,alpha=alpha,entity_out_dim=entity_out_dim,nheads_GAT=nheads_GAT,margin=margin,batch_size_conv=batch_size_conv,alpha_conv=alpha_conv,valid_invalid_ratio_conv=valid_invalid_ratio_conv,out_channels=out_channels,drop_conv=drop_conv)

config.dumps_config()
config

dataset: FB15k
data_folder: ./data
output_folder: ./output
drive_folder: /content/drive/My Drive
save_gdrive: True
cuda: True
epochs_gat: 3000
epochs_conv: 150
weight_decay_gat: 1e-05
weight_decay_conv: 1e-05
pretrained_emb: False
embedding_size: 50
lr: 0.001
get_2hop: True
use_2hop: True
partial_2hop: True
batch_size_gat: 272115
valid_invalid_ratio_gat: 2
drop_GAT: 0.3
alpha: 0.2
entity_out_dim: [100, 200]
nheads_GAT: [2, 2]
margin: 1
batch_size_conv: 128
alpha_conv: 0.2
valid_invalid_ratio_conv: 40
out_channels: 50
drop_conv: 0.3

## 2. WN18RR

In [ ]:
#@title Cấu hình cài đặt WN18RR { run: "auto" }

#@markdown Siêu tham số

dataset = "FB15k" #@param ["WN18RR", "WN18", "FB15k", "FB15k-237", "Thanh"] {allow-input: false}
cuda = True #@param {type: "boolean"}
data_folder = "./data"
output_folder="./output" #@param {type: "string"}
drive_folder="/content/drive/My Drive" #@param {type: "string"}
save_gdrive = True #@param {type: "boolean"}
epochs_gat = 3600 #@param {type: "number"}
epochs_conv = 200 #@param {type: "number"}
weight_decay_gat = float(5e-6) #@param {type:"raw"}
weight_decay_conv = float(1e-5) #@param {type:"raw"}
pretrained_emb = False #@param {type: "boolean"}
embedding_size = 50 #@param {type: "slider", min: 20, max: 200}
lr = float(1e-3) #@param {type:"raw"}
get_2hop = True #@param {type: "boolean"}
use_2hop = True #@param {type: "boolean"}
partial_2hop = False #@param {type: "boolean"}

#@markdown --- Tham số cho mô hình GAT
batch_size_gat = 86835 #@param {type: "number"}
valid_invalid_ratio_gat = 2 #@param {type: "slider", min: 1, max: 10}
drop_GAT = 0.3  #@param {type:"raw"}
alpha = 0.2  #@param {type:"raw"}
entity_out_dim = [100, 200]  #@param {type:"raw"}
nheads_GAT = [2, 2]  #@param {type:"raw"}
margin = 5 #@param {type: "slider", min: 1, max: 10}

#@markdown --- Tham số cho mô hình ConvKB
batch_size_conv = 128  #@param {type: "slider", min: 16, max: 512, step:16}
alpha_conv = 0.2  #@param {type:"raw"}
valid_invalid_ratio_conv = 40
out_channels = 500 #@param {type: "slider", min: 100, max: 1000, step:100}
drop_conv = 0.0  #@param {type:"raw"}

from create_config import Config

config = Config(dataset=dataset,data_folder=data_folder,cuda=cuda,output_folder=output_folder,save_gdrive=save_gdrive,drive_folder=drive_folder,epochs_gat=epochs_gat,epochs_conv=epochs_conv,weight_decay_gat=weight_decay_gat,weight_decay_conv=weight_decay_conv,pretrained_emb=pretrained_emb,embedding_size=embedding_size,lr=lr,get_2hop=get_2hop,use_2hop=use_2hop,partial_2hop=partial_2hop,batch_size_gat=batch_size_gat,valid_invalid_ratio_gat=valid_invalid_ratio_gat,drop_GAT=drop_GAT,alpha=alpha,entity_out_dim=entity_out_dim,nheads_GAT=nheads_GAT,margin=margin,batch_size_conv=batch_size_conv,alpha_conv=alpha_conv,valid_invalid_ratio_conv=valid_invalid_ratio_conv,out_channels=out_channels,drop_conv=drop_conv)

config.dumps_config()

config

dataset: FB15k
data_folder: ./data
output_folder: ./output
drive_folder: /content/drive/My Drive
save_gdrive: True
cuda: True
epochs_gat: 3600
epochs_conv: 200
weight_decay_gat: 5e-06
weight_decay_conv: 1e-05
pretrained_emb: False
embedding_size: 50
lr: 0.001
get_2hop: True
use_2hop: True
partial_2hop: False
batch_size_gat: 86835
valid_invalid_ratio_gat: 2
drop_GAT: 0.3
alpha: 0.2
entity_out_dim: [100, 200]
nheads_GAT: [2, 2]
margin: 5
batch_size_conv: 128
alpha_conv: 0.2
valid_invalid_ratio_conv: 40
out_channels: 500
drop_conv: 0.0

Load config

# II. Create corpus


In [ ]:
!python 1_create_corpus.py

number of unique_entities -> 14951
number of unique_entities -> 13292
number of unique_entities -> 13584
test_triples (59071, 3)
test_adjacency_mat (3, 59071)
entity2id 14951
relation2id 1345
headTailSelector 1345
unique_entities_train (14951,) 

Initialised relations and entities randomly
Graph created
length of graph keys is  14834
time taken  2975.4948360919952
length of neighbors dict is  14808
Total triples count 592213, training triples 483142, validation_triples 50000, test_triples 59071
tcmalloc: large alloc 1073741824 bytes == 0x6cb64000 @  0x7f8a1a12f1e7 0x5acbbb 0x4e05c6 0x4e2022 0x4e1fcb 0x4e241b 0x4e2e4d 0x4e2a5c 0x4e2a04 0x4e0ea8 0x4e263b 0x4e34a6 0x5eb622 0x50a51c 0x50c1f4 0x507f24 0x509c50 0x50a64d 0x50c1f4 0x507f24 0x509c50 0x50a64d 0x50c1f4 0x507f24 0x509c50 0x50a64d 0x50cfd6 0x507f24 0x50b053 0x634dd2 0x634e87
tcmalloc: large alloc 2147483648 bytes == 0xacb64000 @  0x7f8a1a12f1e7 0x5acbbb 0x4e05c6 0x4e2022 0x4e2443 0x4e2e4d 0x4e2b00 0x4e2a04 0x4e0ea8 0x4e263b 0x4e34a

In [ ]:
!cp "/content/drive/My Drive/FB15k_cuda_corpus.pt" "/content/data/FB15k/FB15k_cuda_corpus.pt"
!cp "/content/drive/My Drive/FB15k_cuda_entity_embeddings.pt" "/content/data/FB15k/FB15k_cuda_entity_embeddings.pt"
!cp "/content/drive/My Drive/FB15k_cuda_relation_embeddings.pt" "/content/data/FB15k/FB15k_cuda_relation_embeddings.pt"

## III. Training GAT encoder

In [ ]:
!python 2_training_encoder.py

Streaming output truncated to the last 5000 lines.
Iteration-> 1  , Iteration_time-> 1.9712 , Iteration_loss 0.0186
Epoch 2000 , average loss 0.01869508158415556 , epoch_time 4.350212812423706

epoch->  2001
Iteration-> 0  , Iteration_time-> 2.3646 , Iteration_loss 0.0184
Iteration-> 1  , Iteration_time-> 1.9561 , Iteration_loss 0.0186
Epoch 2001 , average loss 0.018529068678617477 , epoch_time 4.321019649505615

epoch->  2002
Iteration-> 0  , Iteration_time-> 2.3612 , Iteration_loss 0.0185
Iteration-> 1  , Iteration_time-> 1.9534 , Iteration_loss 0.0188
Epoch 2002 , average loss 0.018644485622644424 , epoch_time 4.31484842300415

epoch->  2003
Iteration-> 0  , Iteration_time-> 2.3428 , Iteration_loss 0.0186
Iteration-> 1  , Iteration_time-> 1.9598 , Iteration_loss 0.0184
Epoch 2003 , average loss 0.018452689982950687 , epoch_time 4.302917003631592

epoch->  2004
Iteration-> 0  , Iteration_time-> 2.3535 , Iteration_loss 0.0185
Iteration-> 1  , Iteration_time-> 1.9698 , Iteration_loss 0

# III Training conv decoder

In [ ]:
%%bash
python 3_training_decoder.py

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

tcmalloc: large alloc 1343922176 bytes == 0x5212000 @  0x7f7ac842f1e7 0x7f7ab5eb05b6 0x7f7ac4792ff8 0x7f7ac44709b0 0x566ddc 0x50a783 0x50c1f4 0x507f24 0x588fac 0x59fe1e 0x50d596 0x507f24 0x509c50 0x50a64d 0x50c1f4 0x507f24 0x509c50 0x50a64d 0x50cfd6 0x507f24 0x50b053 0x634dd2 0x634e87 0x63863f 0x6391e1 0x4b0dc0 0x7f7ac802cb97 0x5b26fa
tcmalloc: large alloc 1343922176 bytes == 0x553bc000 @  0x7f7ac842f1e7 0x59221c 0x7f7ac4793057 0x7f7ac44709b0 0x566ddc 0x50a783 0x50c1f4 0x507f24 0x588fac 0x59fe1e 0x50d596 0x507f24 0x509c50 0x50a64d 0x50c1f4 0x507f24 0x509c50 0x50a64d 0x50cfd6 0x507f24 0x50b053 0x634dd2 0x634e87 0x63863f 0x6391e1 0x4b0dc0 0x7f7ac802cb97 0x5b26fa
tcm

# V. Evaluation

### 1. Evaluating

In [ ]:
!python 4_evalution.py

Loading corpus
tcmalloc: large alloc 1343922176 bytes == 0x45f0000 @  0x7fb709a7e1e7 0x7fb6f74ff5b6 0x7fb705de1ff8 0x7fb705abf9b0 0x566ddc 0x50a783 0x50c1f4 0x507f24 0x588fac 0x59fe1e 0x50d596 0x507f24 0x509c50 0x50a64d 0x50c1f4 0x507f24 0x509c50 0x50a64d 0x50cfd6 0x507f24 0x50b053 0x634dd2 0x634e87 0x63863f 0x6391e1 0x4b0dc0 0x7fb70967bb97 0x5b26fa
tcmalloc: large alloc 1343922176 bytes == 0x5479a000 @  0x7fb709a7e1e7 0x59221c 0x7fb705de2057 0x7fb705abf9b0 0x566ddc 0x50a783 0x50c1f4 0x507f24 0x588fac 0x59fe1e 0x50d596 0x507f24 0x509c50 0x50a64d 0x50c1f4 0x507f24 0x509c50 0x50a64d 0x50cfd6 0x507f24 0x50b053 0x634dd2 0x634e87 0x63863f 0x6391e1 0x4b0dc0 0x7fb70967bb97 0x5b26fa
tcmalloc: large alloc 2147483648 bytes == 0xe4944000 @  0x7fb709a7f2a4 0x4dcd26 0x4df1d1 0x50a7b1 0x50c1f4 0x507f24 0x588fac 0x59fe1e 0x50d596 0x507f24 0x509c50 0x50a64d 0x50c1f4 0x507f24 0x509c50 0x50a64d 0x50cfd6 0x507f24 0x50b053 0x634dd2 0x634e87 0x63863f 0x6391e1 0x4b0dc0 0x7fb70967bb97 0x5b26fa
Loading model


### 2. Load result